# Introduction to GPU Programming Summer School
D. Quigley, University of Warwick
## Tutorial 1. Using CUDA accelerated libraries

### Step 1 : Sanity check of Python environment

All being well, you've reading this text inside a Jupyter notebook environment running on an SCRTP machine equipped with a CUDA-compatible GPU card and have launched the notebook from within an environment configured to support GPU computing. If in doubt, check the [connecting instructions](https://warwick.ac.uk/fac/sci/maths/research/events/2017-18/nonsymposium/gpu/connecting).

Let's run some simpple checks to make sure you can execute code on a GPU from within this notebook. For today we'll be working with the Python interface to CUDA provided via the numba package.

Some terminology we need aleady:

**Host**        : The traditional computer in which our code is running on a CPU with access to host RAM.

**CUDA Device** : The GPU card consisting of its own RAM and computing cores (lots of them). 

In [1]:
import platform          # So we can figure out where we're running
from numba import cuda   # Import python interface to CUDA

# Report where we're running
print("========================================================")
print("This notebook is running on ", platform.node())
print("========================================================")

# Test if CUDA is available. If so report on the devices present 
if cuda.is_available():  
    
    # List of CUDA capable devices in this system
    for device in cuda.list_devices():       
        print("Device ID : ", device.id, " : ", device.name.decode())         
    
else:
    print("There doesn't appear to be a CUDA capable device in this system")

This notebook is running on  brigitte.csc.warwick.ac.uk
Device ID :  0  :  Tesla K20c
Device ID :  1  :  NVS 310


Let's select the the most appropriate device and query its [compute capability](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities). CUDA devices have varying compute capability depending on the product range and when they were manufactured. Typically CUDA software will be written to support a minimum compute capability.

[Numba requires a compute capability of 2.0 or higher](https://numba.pydata.org/numba-doc/dev/cuda/overview.html), so we should check this.

In [2]:
my_instance = cuda.select_device(0) # Create a device instance to work with based on device 0 

# The compute capability is stored as a tuple (major, minor) so we're good to go if...
if my_instance.compute_capability[0] >= 2:
    print("The selected device (",my_instance.name.decode(),") has a sufficient compute capability")
else:
    print("The selected device does not have a sufficient compute capability")

The selected device ( Tesla K20c ) has a sufficient compute capability


### Step 2 : "Drop in" replacements for standard numerical libraries via Pyculib 

The computationally intensive part of many scientific codes reduces to standard numerical operations, e.g. manipulation of large matrices/vectors, performing Fourier transforms, dealing with sparse linear algebra etc.

In the traditional High Performance Computing (HPC) realm of compiled C/C++/Fortran code, there exists a suite of standard optimised libraries for such things. The CUDA toolkit provides GPU-enabled versions of these, and numba provides Python interfaces to these through the pyculib package which we'll experiment with below.

The advantage of this approach is that no real knowledge of GPU programming is required, we simply replace calls to standard CPU functions with GPU-accelerated equivalents. The disadvantage is that we only accelerate part of the code, and may suffer performance overheads associated with transering data between host and device every time we use one of the functions.


##### 2(a) cuBLAS

BLAS is the suite of [Basic Linear Algebra Subprograms](http://www.netlib.org/blas/). These come in three levels, for both real and complex data types. 

**Level 1** : Vector-vector operations 

**Level 2** : Matrix-vector operations

**Level 3** : Matrix-matrix operations

On any well-managed HPC system, the local installations of numpy and scipy packages will be built on top of BLAS routines (written in C or Fortran) that have been optimised for the particular hardware in use. Optimised BLAS implementations for CPUs include [OpenBLAS](https://www.openblas.net/), [Atlas](http://math-atlas.sourceforge.net/), [Intel MKL](https://software.intel.com/en-us/mkl) and [AMD ACML](https://developer.amd.com/building-with-acml/).

The CUDA toolkit include [cuBLAS](https://developer.nvidia.com/cublas), a GPU-accelerated BLAS implementation. Let's compare how this performs in comparison to numpy. If you're interested, the numpy implementation on the SCRTP desktops has been built using OpenBLAS, but optimised only for the most common Intel CPU features to ensure compatibility. CPU performace will not be great.

[Documentation for the pyculib interface to cuBLAS](http://pyculib.readthedocs.io/en/latest/cublas.html)

Let's illustrate this with a simple matrix-matrix multiplication example. The BLAS routine `dgemm` (double precision, general matrix-matrix multiply) performs the following operation.

$$ C = \alpha AB + \beta C $$

where $A$, $B$ and $C$ are matrices and $\alpha$ and $\beta$ are scalars. Other specialised routines are available for matrices with particular structure (e.g. banded, tri-diagonal, symmetric) but we won't worry about that today.

In [3]:
import numpy as np
import pyculib.blas as cublas      # Python interface to cuBLAS, cuFFT, cuSPARSE and cuRAND

# Set size of matrix to work with
size = 3

# Create some square matrices and fill them with random numbers
A = np.random.rand(size,size)
B = np.random.rand(size,size)

# Alpha and beta
alpha = 0.5
beta = 2.0

# Perform the operation described above using standard numpy
C_np = alpha * np.matmul(A,B) 

In [4]:
# Query what we need to pass into cublas.gemm
help(cublas.gemm)

Help on function gemm in module pyculib.blas:

gemm(transa, transb, alpha, A, B, beta=0, C=None, stream=None)
    Generalized matrix-matrix multiplication:
    
    C <- alpha*transa(A)*transb(B) + beta*C
    
    'beta' and 'C' are optional on input. Return 'C'.



In [5]:
# The equivalent cuBLAS call needs additional arguments to specify if we want to use the
# matrices as supplied ('N' - no operation) or their transpose ('T'). We'll ignore streams
# for now but might come back to that later if time permits. Arguments for beta and C are optional.
C_cu = cublas.gemm('N', 'N', alpha, A, B)

A lot just happend behind the scenes there. The numpy arrays have been copied from the host RAM into the device memory, the matrix operation has been performed using the CUDA cores on the device, and the result has been copied back into the numpy array C_cu in the host memory.

Let's be proper programmers and check that both computations gave the same result.

In [6]:
# Subtract the two results
C_cu - C_np

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,  -6.93889390e-18],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

The next thing to investigate is whether this cuBLAS function is any faster. Let's try a more substantial matrix and time how long numpy takes to perform the same operation as above.

In [7]:
import time   # For timing 

size = 1000

# Create some square matrices and fill them with random numbers
A = np.random.rand(size,size)
B = np.random.rand(size,size)

# What is the time before we start the operation?
t1 = time.perf_counter()

# Perform the operation described above using standard numpy
C_np = alpha * np.matmul(A,B) 

# What is the time after we finish the operation in milliseconds?
t2 = time.perf_counter()

# Print time taken
print("Matrix multiplication using numpy took : ",1000*(t2-t1)," milliseconds.")

Matrix multiplication using numpy took :  92.39946100569796  milliseconds.


To time how long cuBLAS takes to do the same thing we'll use its own timer function.

In [18]:
# Create two CUDA events
event1 = cuda.event(timing=True)
event2 = cuda.event(timing=True)

# First event before we call gemm
event1.record()

# Do the multiplication
C_cu = cublas.gemm('N', 'N', alpha, A, B)

# Second event after we call gemm
event2.record()

# Elapsed time (in milliseconds)
t = cuda.event_elapsed_time(event1,event2)

# Print time taken
print("Matrix multiplication using cuBLAS took : ",t," milliseconds.")

Matrix multiplication using cuBLAS took :  35.06553649902344  milliseconds.


When I did this using the K20c in brigitte.csc.warwick.ac.uk the GPU was a factor of three faster for 1000x1000 matrices. We'll explore how this scales with matrix size using the GPUs on Tinis a little later.

Recall that `cublas.gemm` is copying data to and from the GPU as well as performing the calculation. We can seperate this out into smaller steps to understand where the time is being spent.

First, let's explicitly copy the arrays into the device memory. Note the convention that arrays which live in the device memory are given variable names prefixed with `d_`. For reference in this section see [Section 3.3 of the documentation on Numba for CUDA GPUs](https://numba.pydata.org/numba-doc/dev/cuda/memory.html).

In [38]:
event1.record() # First event

# Create new device arrays - first copy A and B
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)

# and create a new array on the device to hold C (faster than copying an empty array from the host)
d_C = cuda.device_array((size,size))

# We haven't done anything with these arrays yet. The program may continue past this point
# while the arrays are copying and only wait for the copy to finish at the point when
# we need to use them, i.e. the copy is *asynchronous*. The following ensures the operation is
# complete before we record the next timing event.
cuda.synchronize()

event2.record() # Second event

# Elapsed time (in milliseconds)
t = cuda.event_elapsed_time(event1,event2)

# Print time taken
print("Creating device arrays took : ",t," milliseconds.")

Creating device arrays took :  6.630015850067139  milliseconds.


The python interface we're using is clever enough to understand that when we pass device arrays as arguments to cuBLAS functions it should use those instead of copying data over from the host.

In [39]:
# First event before we call gemm
event1.record()

# Do the multiplication
cublas.gemm('N', 'N', alpha, d_A, d_B)

# Second event after we call gemm
event2.record()

# Elapsed time (in milliseconds)
t = cuda.event_elapsed_time(event1,event2)

# Print time taken
print("Matrix multiplication using cuBLAS took : ",t," milliseconds.")

Matrix multiplication using cuBLAS took :  38.06185531616211  milliseconds.


We might now want to copy the result stored in the device array back onto the host.

In [41]:
# First event before we transfer the data
event1.record()

C_cu = d_C.copy_to_host()
cuda.synchronize()

event2.record()

# Elapsed time (in milliseconds)
t = cuda.event_elapsed_time(event1,event2)

# Print time taken
print("Copying output back to host memory took : ",t," milliseconds.")

Copying output back to host memory took :  7.132800102233887  milliseconds.


You'll likely see that the total time taken in the three steps above is a bit longer than when we let the cuBLAS routine handle the copying behind the scenes due to the enforced synchronisation.

The take home point here is that time spent copying data to and from the device takes time. Best performance will be acheived by doing as much work as possible on the arrays while they are on the device, minimising copies between host and device memory.

##### 2(a) Exercises

You should now have everything you need to canabalise the code snippets above and construct some python scripts which you can run outside of this notebook environment on the *Tinis GPU node*. **Please do not run matrices larger than 1000x1000 in this notebook**. Questions to explore:

* How large a matrix is needed before the multiplication is 10x faster on the GPU compared to numpy code? 
* What is the largest speedup you can obtain?
* How does the speedup from BLAS level 1 and 2 routines compare to BLAS level 3?
* What is the variability in your timings, what gives rise to this?
* (Advanced) Is the numpy code making use of multiple CPU cores on the Tinis node?
* (Advanced) How does the cuBLAS/numpy performance compare to using scipy.linalg.blas.dgemm?
* (Very Advanced) How does the shape of the matrix change performance. Why?

Tutors will be available during the session to help you answer these questions.

##### 2(a) cuFFT

